In [ ]:
import numpy as np
from skimage import io

import tqdm
import napari
import pandas as pd

import zarr

import os
import pickle

### Make data for training
Assume training data is double color. One for the channel and another for Cytosol/nuclei. \
N

In [ ]:
# image path
fix_n5_path = '/mnt/ampa02_data01/tmurakami/240417_whole_4color_1st_M037-3pb/fused/fused.n5' # zarr with pyramid resolution
# save path
save_path = '/mnt/ampa02_data01/tmurakami/240417_whole_4color_1st_M037-3pb/model_training/crops'
# metadata path
meta_path = '/mnt/ampa02_data01/tmurakami/240417_whole_4color_1st_M037-3pb/model_training/info.pkl'

# create Zarr file object
fix_zarr = zarr.open(store=zarr.N5Store(fix_n5_path), mode='r')
# if you use ngff ome.zarr
# mov_zarr_path = '/mnt/ampa02_data01/tmurakami/240417_whole_4color_1st_M037-3pb/registration/round02.zarr'
# mov_zarr = zarr.open(mov_zarr_path, mode='r')

n5_setups = list(fix_zarr.keys())

voxel_size = (2.0,1.3,1.3)

In [ ]:
# set your parameters here
reference_chan = 3 # Number or None
segment_chan = 1

corner_positions = [
    [1235,2510,775],
    [1325,3350,1543],
    [1725,4632,1604],
    [2262,6207,2682],
    [1708,3877,2563]
]

crop_size = [100,100,100]

In [ ]:
if not os.path.exists(meta_path):
    df = pd.DataFrame(columns=['ID', 'integer_ID', 'instance_counts', 'corner', 'source', 'ref_channel', 'channel', 'crop_size'])
else:
    df = pd.read_pickle(meta_path)

for i, pos in enumerate(corner_positions):
    idx = len(df)
    
    # find out any duplication between the current data and the metadata
    # if it is duplicated, ask 
    flag = False
    if df['corner'].isin([pos]).any():
        for k in df['integer_ID'][df['corner'].isin([pos])].to_list():
            if ((df.loc[k,'source'] == fix_n5_path) and 
                (df.loc[k,'ref_channel'] == reference_chan) and 
                (df.loc[k,'channel'] == segment_chan) and 
                (df.loc[k,'crop_size'] == crop_size)):
                flag = True
                idx = k
    if flag:
        ans = input("Do you want to re-analyze the data? y or n")
        if ans != 'y':
            continue
        
    # set file path to be saved for both image and mask
    prefix = str(idx)
    while len(prefix) < 4:
        prefix = '0' + prefix
    img_path = os.path.join(save_path, prefix+'_img.tif')
    mask_path = os.path.join(save_path, prefix+'_mask.tif')

    # get the image of a reference of channel
    img_crop_stack = []
    if reference_chan is not None:
        img = fix_zarr[n5_setups[reference_chan]]['timepoint0']['s0']
        img_crop = img[tuple(slice(i,i+j) for i,j in zip(pos, crop_size))]
        img_crop_stack.append(img_crop)

    # get the image of a reference of channel
    img = fix_zarr[n5_setups[segment_chan]]['timepoint0']['s0']
    img_crop = img[tuple(slice(i,i+j) for i,j in zip(pos, crop_size))]
    img_crop_stack.append(img_crop)

    img_crop_stack = np.stack(img_crop_stack)

    # open Napari. Pause for loop until close the window
    viewer = napari.Viewer()
    viewer.add_image(img_crop_stack, channel_axis=0, scale=voxel_size)
    labels = viewer.add_labels(np.zeros_like(img_crop), name='segmentation', scale=voxel_size)
    viewer.show(block=True)

    # save images and segmentation.
    img = np.swapaxes(img_crop_stack,0,1)
    labels_img = labels.data
    io.imsave(img_path, img, plugin='tifffile', imagej=True, metadata={'axes': 'ZCYX'})
    io.imsave(mask_path, labels_img, plugin='tifffile')

    # update the metadata
    df.loc[idx,'ID'] = prefix
    df.loc[idx,'integer_ID'] = idx
    count = (np.unique(labels_img)).size - 1
    df.loc[idx,'instance_counts'] = count
    df.loc[idx,'corner'] = pos
    df.loc[idx, 'source'] = fix_n5_path
    df.loc[idx, 'ref_channel'] = reference_chan
    df.loc[idx, 'channel'] = segment_chan
    df.loc[idx, 'crop_size'] = crop_size

df.to_pickle(meta_path)

In [ ]:
df